<a href="https://colab.research.google.com/github/tomaszwk/NLP/blob/main/Desafio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [2]:
import os
import requests

# URL del libro en formato txt desde el Proyecto Gutenberg
url = 'https://www.gutenberg.org/files/2701/2701-0.txt'  # Ejemplo con "Moby Dick" de Herman Melville

# Nombre del archivo donde se guardará el libro
filename = 'moby_dick.txt'

# Verificar si el archivo ya existe
if not os.path.exists(filename):
    print("Descargando el libro desde el Proyecto Gutenberg...")
    response = requests.get(url)

    if response.status_code == 200:
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f'El libro se ha descargado y guardado como {filename}')
    else:
        print("Hubo un problema al descargar el libro.")
else:
    print(f"El archivo '{filename}' ya existe.")



El archivo 'moby_dick.txt' ya existe.


In [3]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
import nltk
import pandas as pd

# Descargar el tokenizador de oraciones
nltk.download('punkt')

# Leer el contenido del archivo 'moby_dick.txt'
with open('moby_dick.txt', 'r', encoding='utf-8') as file:
    text = file.read()

# Separar el texto en oraciones
sentences = nltk.sent_tokenize(text)

# Crear un DataFrame con cada oración como una fila
df = pd.DataFrame(sentences, columns=['Sentence'])
df.head()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Sentence
0,﻿The Project Gutenberg eBook of Moby-Dick; or ...
1,"You may copy it, give it away or re-use it und..."
2,"If you are not located in the United States, y..."
3,Title: Moby-Dick; or The Whale\n\nAuthor: Herm...
4,By Herman Melville\n\n\n\nCONTENTS\n\nETYMOLOGY.


In [4]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 9181


### 1 - Preprocesamiento

In [5]:
pip install Keras==2.12.0


  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: Keras
    Found existing installation: keras 2.10.0
    Uninstalling keras-2.10.0:
      Successfully uninstalled keras-2.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.10.0 requires keras<2.11,>=2.10.0, but you have keras 2.12.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.10.0 which is incompatible.


In [6]:
pip install tensorflow==2.10.0

  Using cached keras-2.10.0-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.10.0 which is incompatible.


In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-7-4bb61bf6ad25>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


### 2 - Crear los vectores (word2vec)

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=4,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [11]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 9181


In [12]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 4381


### 3 - Entrenar embeddings

In [13]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 504874.53125
Loss after epoch 1: 414125.96875
Loss after epoch 2: 379576.75
Loss after epoch 3: 360688.0
Loss after epoch 4: 353034.125
Loss after epoch 5: 320481.375
Loss after epoch 6: 308732.25
Loss after epoch 7: 307454.0
Loss after epoch 8: 306179.5
Loss after epoch 9: 301542.25
Loss after epoch 10: 297084.25
Loss after epoch 11: 299147.5
Loss after epoch 12: 277231.5
Loss after epoch 13: 269809.5
Loss after epoch 14: 263853.5
Loss after epoch 15: 265168.5
Loss after epoch 16: 261875.0
Loss after epoch 17: 261576.5
Loss after epoch 18: 259738.0
Loss after epoch 19: 257846.5
Loss after epoch 20: 256516.5
Loss after epoch 21: 252539.5
Loss after epoch 22: 255171.0
Loss after epoch 23: 249330.5
Loss after epoch 24: 247070.0
Loss after epoch 25: 245017.5
Loss after epoch 26: 249091.0
Loss after epoch 27: 245378.0
Loss after epoch 28: 235628.5
Loss after epoch 29: 226839.0
Loss after epoch 30: 229077.0
Loss after epoch 31: 224223.0
Loss after epoch 32: 223696.0
Loss

(14135305, 22077100)

### 4 - Ensayar

In [14]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["whale"], topn=10)

[('“hast', 0.3810027837753296),
 ('killer', 0.33427926898002625),
 ('designated', 0.32033321261405945),
 ('beale’s', 0.31745684146881104),
 ('feat', 0.3078577220439911),
 ('v', 0.3053145706653595),
 ('scoresby', 0.3053103983402252),
 ('frederick', 0.3043726682662964),
 ('authorities', 0.29904332756996155),
 ('technically', 0.29787224531173706)]

In [15]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('stepping', 0.05092572420835495),
 ('during', 0.050820983946323395),
 ('thigh', 0.047357380390167236),
 ('caw', 0.04567653313279152),
 ('subject', 0.04554687440395355),
 ('links', 0.04429538547992706),
 ('beneath', 0.04298143461346626),
 ('slender', 0.04060402140021324),
 ('wholly', 0.03290611878037453),
 ('means', 0.030932491645216942)]

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('goin’', 0.37935659289360046),
 ('twenty', 0.37709060311317444),
 ('years’', 0.36710086464881897),
 ('five', 0.35121291875839233),
 ('ridge', 0.34427475929260254),
 ('sixty', 0.3423555791378021),
 ('eighteen', 0.3414296805858612),
 ('six', 0.33636417984962463),
 ('yards', 0.32665517926216125),
 ('sorts', 0.3192387819290161)]

In [40]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["moby"], topn=5)

[('dick', 0.7595923542976379),
 ('“god', 0.37841933965682983),
 ('earlier', 0.3711804747581482),
 ('mile', 0.36969640851020813),
 ('bless', 0.35548901557922363)]

In [24]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["whale"], topn=10)

[('“hast', 0.3810027837753296),
 ('killer', 0.33427926898002625),
 ('designated', 0.32033321261405945),
 ('beale’s', 0.31745684146881104),
 ('feat', 0.3078577220439911),
 ('v', 0.3053145706653595),
 ('scoresby', 0.3053103983402252),
 ('frederick', 0.3043726682662964),
 ('authorities', 0.29904332756996155),
 ('technically', 0.29787224531173706)]

In [43]:
# el método `get_vector` permite obtener los vectores:
vector_whale = w2v_model.wv.get_vector("whale")
print(vector_whale)

[-2.20049694e-01  5.45216560e-01  1.46163300e-01  1.59049287e-01
 -1.15445420e-01 -2.84355413e-02  3.44187438e-01  4.97637868e-01
  2.27356359e-01 -1.67559370e-01  2.96008795e-01 -1.76022470e-01
 -1.45135261e-03  1.21515676e-01  6.89677447e-02 -4.49861959e-02
  5.42619109e-01 -5.19128561e-01 -3.78556773e-02 -4.00886685e-01
  1.96594149e-01 -1.18480492e-02 -5.80869541e-02 -3.15241724e-01
  1.71946019e-01 -2.42200717e-01 -9.37238336e-02 -1.45085886e-01
 -2.79885918e-01  1.01333849e-01  5.57826981e-02 -2.01355577e-01
 -3.11721981e-01 -1.61708444e-01 -3.65316361e-01  8.12738091e-02
  2.35225201e-01  1.88330904e-01  3.01481247e-01 -1.15861468e-01
  2.00999364e-01 -7.32027888e-02  2.98784822e-01 -3.71304527e-03
  1.01741530e-01 -4.96414537e-03 -3.92204404e-01  2.62895763e-01
 -1.97127625e-01 -9.52214748e-02  1.56607665e-02 -3.29372212e-02
 -4.52129841e-01  6.94828391e-01  1.23917960e-01 -6.70075178e-01
  3.50955367e-01 -2.07593262e-01 -3.46151851e-02  2.52562433e-01
  5.66831566e-02 -2.09101

In [44]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_whale)

[('whale', 1.0),
 ('“hast', 0.3810027539730072),
 ('killer', 0.33427926898002625),
 ('designated', 0.32033321261405945),
 ('beale’s', 0.31745684146881104),
 ('feat', 0.3078577220439911),
 ('v', 0.3053145706653595),
 ('scoresby', 0.3053103983402252),
 ('frederick', 0.3043726682662964),
 ('authorities', 0.29904329776763916)]

In [35]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["animals"], topn=10)

[('fossil', 0.46316736936569214),
 ('cattle', 0.4487903118133545),
 ('damned', 0.4482507109642029),
 ('standers', 0.4386444389820099),
 ('talking', 0.4383048713207245),
 ('respect', 0.4269738495349884),
 ('seams', 0.41736650466918945),
 ('hercules', 0.4127228856086731),
 ('compare', 0.40987613797187805),
 ('orphans', 0.4098072648048401)]

### 5 - Visualizar agrupación de vectores

In [28]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)


    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [29]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px
#
vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

el objetivo en este caso fue el de evaluar la similitud de palabras dentro del libro Moby - dick obtenido desde https://www.gutenberg.org/ , lo que vemos es que si bien hay palabras que encuentra una similitud razonables hay otras que no , y esto tambien esta explicado porque al momento de entrenar , a pesar de utilizar 100 epocas de entrenamiento , seguimos encontrando una metrica de perdida alta.


### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.